In [1]:
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.image as img
import matplotlib.pyplot as plt

In [2]:
path = 'C:/Users/JGUILHERME/Documents/UFSC/Projeto Intel/RNN/DATASET_WORDS/letters_train/'

batch_size = 64

c2n = {'PAD': 0,'GO': 1,'EOS': 2,'a': 3,'A': 4,'b': 5,'B': 6,'c': 7,'C': 8,'d': 9,'D': 10,'e': 11,'E': 12,'f': 13,
       'F': 14,'g': 15,'G': 16,'h': 17,'H': 18,'i': 19,'I': 20,'j': 21,'J': 22,'k': 23,'K': 24,'l': 25,'L': 26,'m': 27,
       'M': 28,'n': 29,'N': 30,'o': 31,'O': 32,'p': 33,'P': 34,'q': 35,'Q': 36,'r': 37,'R': 38,'s': 39,'S': 40,'t': 41,
       'T': 42,'u': 43,'U': 44,'v': 45,'V': 46,'w': 47,'W': 48,'x': 49,'X': 50,'y': 51,'Y': 52,'z': 53,'Z': 54,'-': 55}

vocab_size = len(c2n) 

In [3]:
def aplly_dict(letter):        
        j = c2n[letter]
        return j
    
def get_pic_name(pic):
        p = pic.find('(')
        if(p != -1):
            word = str(pic[:(p-1)])
        else:
            p = pic.find('.')
            word = str(pic[:p])
        return word

def aplly_one_hot(index):
        one_hot = np.zeros(shape=[vocab_size], dtype=np.float32)        
        one_hot[index] = 1
        return one_hot
    

folders = os.listdir(path)

i = 0
for folder in folders:
    pics = os.listdir(path+folder+'/')
    for pic in pics:
        #tratamento Y
        picture = get_pic_name(pic)
        index = aplly_dict(picture)
        one_hot = aplly_one_hot(index)
        #tratamento X
        photo = img.imread(path+folder+'/'+pic)
        if(i != 1):
            y_arrange = [one_hot]   
            x_arrange = [photo]
            i = 1
        else:
            y_arrange = np.append(y_arrange, [one_hot], axis=0)
            x_arrange = np.append(x_arrange, [photo], axis=0)        
        
print(np.shape(y_arrange))
print(np.shape(x_arrange))

X_train, X_test, y_train, y_test = train_test_split(x_arrange, y_arrange, test_size=0.2)

print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))



(2337, 56)
(2337, 50, 50)
(1869, 50, 50)
(468, 50, 50)
(1869, 56)
(468, 56)


In [4]:
tf.reset_default_graph()

tr_data = tf.contrib.data.Dataset.from_tensor_slices((x_arrange,y_arrange))

batched_train = tf.contrib.data.Dataset.batch(tr_data, batch_size=batch_size)

iterator = tf.contrib.data.Iterator.from_structure(batched_train.output_types,batched_train.output_shapes)

next_element = iterator.get_next()

train_init = iterator.make_initializer(batched_train)

In [5]:
#placeholders
x = tf.placeholder(dtype=tf.float32, shape=[None, 50, 50])
y = tf.placeholder(dtype=tf.float32, shape=[None, vocab_size])

x_reshape = tf.reshape(tensor=x, shape=[-1, 50, 50, 1])

#CNN
filters1 = 64 
filters2 = 64
stride1 = 2
stride2 = 2

conv1_1 = tf.layers.conv2d(inputs=x_reshape, activation=tf.nn.relu, filters=filters1, kernel_size=[5,5], padding='same', use_bias=True, trainable=True)
conv1_2 = tf.layers.conv2d(inputs=conv1_1, activation=tf.nn.relu, filters=filters1, kernel_size=[5,5], padding='same', use_bias=True, trainable=True)
pool1 = tf.layers.max_pooling2d(inputs=conv1_2, pool_size=[2,2], strides=stride1)

conv2_1 = tf.layers.conv2d(inputs=pool1, activation=tf.nn.relu, filters=filters2, kernel_size=[5,5], padding='same', use_bias=True, trainable=True)
conv2_2 = tf.layers.conv2d(inputs=conv2_1, activation=tf.nn.relu, filters=filters2, kernel_size=[5,5], padding='same', use_bias=True, trainable=True)
pool2 = tf.layers.max_pooling2d(inputs=conv2_2, pool_size=[2,2], strides=stride2)

conv3_1 = tf.layers.conv2d(inputs=pool2, activation=tf.nn.relu, filters=filters2, kernel_size=[5,5], padding='same', use_bias=True, trainable=True)
conv3_2 = tf.layers.conv2d(inputs=conv3_1, activation=tf.nn.relu, filters=filters2, kernel_size=[5,5], padding='same', use_bias=True, trainable=True)
pool3 = tf.layers.max_pooling2d(inputs=conv3_2, pool_size=[2,2], strides=stride2)

flatten = tf.reshape(tensor=pool3, shape=[-1, filters2 * 6 * 6])

dense1 = tf.layers.dense(inputs=flatten, activation=tf.nn.relu, units=512, trainable=True, use_bias=True)
drop1 = tf.layers.dropout(inputs=dense1, rate=0.3,training=True)
dense2 = tf.layers.dense(inputs=drop1, units=vocab_size, trainable=True, use_bias=True)

pred = tf.nn.softmax(dense2)

In [6]:
cost = tf.losses.softmax_cross_entropy(onehot_labels=y, logits= dense2)
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= dense2, labels= y))
optimizer = tf.train.AdamOptimizer(0.001). minimize(cost)

correct_prediction = tf.equal(tf.argmax(dense2,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver()

In [7]:
epochs = 50
with tf.Session() as sess:
    sess.run([train_init, tf.global_variables_initializer()])
    
    for i in range(epochs):
        print('epoch: ', i)
        while(True):
            try:
                a,b = sess.run(next_element)
            except:
                sess.run(train_init)
                break
            
            loss, _ = sess.run([cost, optimizer], feed_dict={x:a, y:b})
            print('loss: ', loss)
        
        acc = sess.run(accuracy, feed_dict={x: X_test, y: y_test})
        print('accuracy: ', 100*acc)
    
    saver.save(sess, "./cnn_letters") #salva as matrizes treinadas

epoch:  0
loss:  4.02373
loss:  2.92292
loss:  15.3352
loss:  4.47646
loss:  4.10068
loss:  4.0266
loss:  4.02483
loss:  4.01434
loss:  4.01997
loss:  4.0262
loss:  4.0137
loss:  4.02098
loss:  4.0271
loss:  4.00778
loss:  4.01606
loss:  4.02533
loss:  4.01156
loss:  4.0094
loss:  4.02344
loss:  4.00758
loss:  4.00547
loss:  4.02286
loss:  3.99633
loss:  4.00674
loss:  4.01938
loss:  3.99779
loss:  4.01497
loss:  4.01524
loss:  3.97762
loss:  4.00691
loss:  4.0143
loss:  3.96773
loss:  3.9929
loss:  4.00774
loss:  3.95945
loss:  4.00666
loss:  4.0046
accuracy:  4.91452999413
epoch:  1
loss:  3.9184
loss:  3.92097
loss:  3.86488
loss:  3.9121
loss:  3.93227
loss:  3.89191
loss:  3.94007
loss:  3.71828
loss:  3.9513
loss:  4.01374
loss:  3.47736
loss:  4.04904
loss:  4.02026
loss:  3.41073
loss:  3.67934
loss:  3.98008
loss:  3.69659
loss:  3.61443
loss:  3.87629
loss:  3.6511
loss:  3.51449
loss:  3.97949
loss:  3.44736
loss:  3.62386
loss:  3.90452
loss:  3.6182
loss:  3.90265
loss:  3

loss:  1.83677
loss:  2.49158
loss:  1.73041
loss:  2.40779
loss:  2.64143
loss:  3.56398
loss:  2.59292
loss:  2.6356
loss:  2.1679
loss:  2.79058
loss:  2.48687
loss:  2.79611
loss:  2.72118
loss:  2.63475
loss:  2.42319
loss:  2.66536
loss:  2.5007
loss:  2.43921
loss:  2.62203
loss:  2.73274
loss:  1.96641
loss:  2.38434
loss:  2.66276
loss:  2.53231
loss:  2.75283
loss:  2.13935
loss:  2.56938
loss:  2.19164
loss:  1.64164
loss:  2.26307
loss:  2.0005
loss:  1.93941
loss:  2.21176
loss:  1.79248
loss:  1.6483
loss:  2.28761
loss:  1.64855
accuracy:  43.8034176826
epoch:  15
loss:  1.41246
loss:  2.02635
loss:  1.41869
loss:  1.72274
loss:  2.39473
loss:  3.37334
loss:  2.39266
loss:  2.62948
loss:  2.03244
loss:  2.81119
loss:  1.99439
loss:  2.52538
loss:  2.33545
loss:  2.40301
loss:  2.17035
loss:  2.05392
loss:  2.12368
loss:  2.1199
loss:  2.22092
loss:  2.43053
loss:  2.15871
loss:  1.99246
loss:  2.09628
loss:  2.31518
loss:  2.08445
loss:  1.90007
loss:  2.29167
loss:  1.7

loss:  0.403847
loss:  0.144669
loss:  0.105889
loss:  0.0842133
loss:  0.50266
loss:  0.231089
loss:  0.200495
loss:  0.0643563
loss:  0.158221
loss:  0.209909
loss:  0.110626
loss:  0.22908
loss:  0.339347
loss:  0.195042
loss:  0.194574
loss:  0.225863
loss:  0.125167
loss:  0.410917
loss:  0.203801
accuracy:  92.5213694572
epoch:  28
loss:  0.511518
loss:  0.467724
loss:  0.142577
loss:  0.394718
loss:  0.412028
loss:  0.272104
loss:  0.180109
loss:  0.236313
loss:  0.0747911
loss:  0.130412
loss:  0.148434
loss:  0.109581
loss:  0.188915
loss:  0.212201
loss:  0.185331
loss:  0.365535
loss:  0.239198
loss:  0.265922
loss:  0.248327
loss:  0.162693
loss:  0.344735
loss:  0.170318
loss:  0.121273
loss:  0.363215
loss:  0.172345
loss:  0.1438
loss:  0.192429
loss:  0.107205
loss:  0.0474859
loss:  0.297004
loss:  0.200375
loss:  0.182269
loss:  0.314234
loss:  0.0665475
loss:  0.0609973
loss:  0.279586
loss:  0.181377
accuracy:  91.8803393841
epoch:  29
loss:  0.585092
loss:  0.19998

loss:  0.0351954
loss:  0.0399806
loss:  0.00584931
loss:  0.0046463
loss:  0.0195657
loss:  0.013488
loss:  0.0713998
loss:  0.0522335
loss:  0.0316714
loss:  0.029006
loss:  0.110439
loss:  0.0482992
loss:  0.0165027
loss:  0.0299607
loss:  0.0981415
loss:  0.0183692
loss:  0.0105882
loss:  0.0202316
loss:  0.0250204
loss:  0.012925
loss:  0.00271837
loss:  0.0116415
loss:  0.0477798
loss:  0.0302305
loss:  0.0135289
loss:  0.0161306
loss:  0.0304161
loss:  0.0302398
loss:  0.00620056
loss:  0.0245974
loss:  0.000108523
accuracy:  100.0
epoch:  41
loss:  0.00947106
loss:  0.00340582
loss:  0.0132099
loss:  0.0503729
loss:  0.116025
loss:  0.00196153
loss:  0.007725
loss:  0.0752748
loss:  0.325219
loss:  0.02054
loss:  0.0103164
loss:  0.0062228
loss:  0.0171179
loss:  0.0800489
loss:  0.0143379
loss:  0.0083535
loss:  0.0125748
loss:  0.0391755
loss:  0.0165953
loss:  0.0268668
loss:  0.0435789
loss:  0.0262589
loss:  0.0287396
loss:  0.0139438
loss:  0.0979607
loss:  0.00447551
los